In [8]:
# Importing relevant librairies
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import os
pd.set_option('display.max_columns', None)

# Importing project librairies
from dynamic_portfolio.utils import load_csv, return_tickers

In [19]:
test = load_csv('T')

KeyError: 'date'

In [16]:
test.head()

,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,reportedEPS,surprisePercentage,10Y_yield,2Y_yield,10_2_spread,oil_price,orders,fed_funds,unemployment_rate,inf_exp,non_farm_payroll,CPI,retail_sales,gdp_per_capita,gold_price,usd_price,return
0,2000-01-19,105.62,108.75,103.37,106.56,0.810916,5336100.0,0.0,1.0,0.04,33.3333,0.0673,0.0646,0.0027,29.11,185813.0,0.0547,4.0,0.03,128992.0,168.8,213709.0,45983.0,289.15,101.760002,0.000000
1,2000-01-20,115.50,121.50,113.50,113.50,0.863729,16349400.0,0.0,1.0,0.04,0.0000,0.0679,0.0649,0.0030,29.67,185813.0,0.0544,4.0,0.03,128992.0,168.8,213709.0,45983.0,288.25,101.360001,0.065128
2,2000-01-21,114.25,114.25,110.19,111.31,0.847063,4427900.0,0.0,1.0,0.04,0.0000,0.0679,0.0648,0.0031,29.71,185813.0,0.0536,4.0,0.03,128992.0,168.8,213709.0,45983.0,288.70,101.760002,-0.019295
3,2000-01-24,108.44,112.75,105.12,106.25,0.808557,3936400.0,0.0,1.0,0.04,0.0000,0.0669,0.0643,0.0026,29.25,185813.0,0.0553,4.0,0.03,128992.0,168.8,213709.0,45983.0,287.40,102.199997,-0.045459
4,2000-01-25,105.00,113.12,102.37,112.25,0.854216,4438800.0,0.0,1.0,0.04,0.0000,0.0670,0.0644,0.0026,30.28,185813.0,0.0546,4.0,0.03,128992.0,168.8,213709.0,45983.0,285.60,102.410004,0.056471


In [17]:
test.dtypes

date                   object
open                  float64
high                  float64
low                   float64
close                 float64
adjusted_close        float64
volume                float64
dividend_amount       float64
split_coefficient     float64
reportedEPS           float64
surprisePercentage    float64
10Y_yield             float64
2Y_yield              float64
10_2_spread           float64
oil_price             float64
orders                float64
fed_funds             float64
unemployment_rate     float64
inf_exp               float64
non_farm_payroll      float64
CPI                   float64
retail_sales          float64
gdp_per_capita        float64
gold_price            float64
usd_price             float64
return                float64
dtype: object

In [18]:
test.describe()

,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,reportedEPS,surprisePercentage,10Y_yield,2Y_yield,10_2_spread,oil_price,orders,fed_funds,unemployment_rate,inf_exp,non_farm_payroll,CPI,retail_sales,gdp_per_capita,gold_price,usd_price,return
count,5686.000000,5686.000000,5686.000000,5686.000000,5686.000000,5.686000e+03,5686.000000,5686.000000,5686.000000,5686.000000,5686.000000,5686.000000,5686.000000,5686.000000,5686.000000,5686.000000,5686.000000,5686.000000,5686.000000,5686.000000,5686.000000,5686.000000,5686.000000,5686.000000,5686.000000
mean,174.621932,176.561332,172.561271,174.619041,28.284243,3.114785e+07,0.007272,1.001935,0.380412,0.206844,0.032032,0.019786,0.012247,62.610828,211847.780865,0.016258,5.876855,0.030732,138138.218431,222.863775,365028.966585,52238.541154,1045.390439,91.461922,0.001245
std,153.561511,154.840175,152.083381,153.500137,42.443611,2.979771e+07,0.118416,0.090905,0.436595,2.837494,0.013295,0.016940,0.009068,26.197311,31611.606609,0.018617,1.958151,0.007450,7086.577807,30.922248,90639.301243,3765.951125,519.187124,11.308586,0.025020
min,12.990000,13.190000,12.720000,13.120000,0.199685,7.025000e+05,0.000000,1.000000,-0.009700,-10.135100,0.005200,0.000900,-0.007100,-36.980000,137245.000000,0.000400,3.500000,0.004000,127804.000000,168.800000,213709.000000,45983.000000,255.300000,71.330002,-0.518692
25%,68.170000,69.057500,66.880000,67.922500,1.619168,1.137652e+07,0.000000,1.000000,0.047500,0.000000,0.021300,0.005600,0.003900,41.542500,187904.000000,0.001300,4.400000,0.027000,131930.000000,197.600000,298636.000000,49886.000000,465.837500,81.757500,-0.010346
50%,130.355000,132.255000,128.520000,130.320050,10.802164,2.283845e+07,0.000000,1.000000,0.250000,0.000000,0.029800,0.015200,0.012700,59.710000,214872.000000,0.010000,5.400000,0.030000,136462.000000,225.722000,346620.000000,51684.000000,1193.275000,90.610001,0.000887
75%,206.072500,208.669925,204.000000,207.087500,32.681519,3.874406e+07,0.000000,1.000000,0.550000,0.000000,0.042500,0.029500,0.019700,83.620000,232661.000000,0.023700,6.700000,0.032000,144046.000000,243.801000,413203.000000,54714.000000,1396.225000,97.637499,0.013184
max,702.410000,705.070000,699.570000,702.100000,180.959680,3.326072e+08,3.290000,7.000007,2.100000,100.000000,0.067900,0.069300,0.029100,145.310000,294470.000000,0.068600,14.700000,0.054000,154369.000000,298.012000,632849.000000,60154.000000,2063.187500,120.900002,0.136887


In [32]:
def features_creation(ticker: str, high_low_ratio: bool = True, volatility: bool = True, momentum: bool = True, distance: bool = True, volume: bool = True, price_eps_ratio: bool = True,
                      momentum_eps_ratio: bool = True, gold_return: bool = True, oil_return: bool = True, usd_return: bool = True, cpi_return: bool = True, period:int = 250,
                      gdp_return: bool = True, days:list = [5, 10, 20] ):
    """
    Function that adds features to a dataframe
    days should be a list containing the number of days to consider for calculating the volatitliy, momentum, distance and and custom volume
    in order to calculate the momentum/eps ratio, momentum should be True
    """

    final_df = load_csv(ticker=ticker)

    if high_low_ratio==True:
        final_df['high/low'] = final_df['high']/final_df['low'] - 1 # max variation in %

    if volatility==True:
        for day in days:
            #sinon creer une variable du nom de ma colonne
            final_df[f'volatility_{day}days'] = final_df['return'].rolling(day).std().shift(1)

    if momentum==True:
        for day in days:

            final_df[f'momentum_{day}days'] = final_df['return'].rolling(day).mean().shift(1)

    if distance==True:
        for day in days:

            final_df[f'distance_{day}days'] = (final_df['return'] - final_df['return'].rolling(day).mean().shift(1))

    if volume==True:
        for day in days:

            final_df[f'volume_{day}days'] = final_df['volume'].rolling(day).mean().shift(1)

    if price_eps_ratio==True:
        final_df['price/eps'] = final_df['adjusted_close']/final_df['reportedEPS']

#     if momentum_eps_ratio==True:
#         final_df['momentum/eps'] = final_df['momentum']/final_df['reportedEPS']

    if gold_return==True:
        final_df['gold_return'] = final_df['gold_price'].pct_change()

    if oil_return==True:
        final_df['oil_return'] = final_df['oil_price'].pct_change()

    if usd_return==True:
        final_df['usd_return'] = final_df['usd_price'].pct_change()

    if cpi_return==True:
        final_df['cpi_return'] = final_df['CPI'].pct_change(periods=period)

    if gdp_return==True:
        final_df['gdp_return'] = final_df['gdp_per_capita'].pct_change(periods=period)
        
    final_df.dropna(inplace=True)

    return final_df


In [33]:
features_creation('AAPL')

/home/racho961/code/dynamic_portfolio/dynamic_portfolio/utils.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['return'][0]=0


,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,reportedEPS,surprisePercentage,10Y_yield,2Y_yield,10_2_spread,oil_price,orders,fed_funds,unemployment_rate,inf_exp,non_farm_payroll,CPI,retail_sales,gdp_per_capita,gold_price,usd_price,return,high/low,volatility_5days,volatility_10days,volatility_20days,momentum_5days,momentum_10days,momentum_20days,distance_5days,distance_10days,distance_20days,volume_5days,volume_10days,volume_20days,price/eps,gold_return,oil_return,usd_return,cpi_return,gdp_return
250,2001-01-18,17.810,18.75,17.620,18.69,0.284460,21910900.0,0.0,1.0,-0.0097,0.0,0.0512,0.0472,0.0040,30.42,167108.0,0.0593,4.2,0.03,130668.0,175.100,226791.0,46519.0,264.8000,109.940002,0.111838,0.064132,0.052955,0.052157,0.040465,-0.003374,0.013462,0.009728,0.115213,0.098376,0.102110,10553160.0,10511320.0,8720080.00,-29.325744,0.005124,0.021834,-0.007851,0.037322,0.011656
251,2001-01-19,19.440,19.56,18.690,19.50,0.296788,13869000.0,0.0,1.0,-0.0097,0.0,0.0519,0.0475,0.0044,32.12,167108.0,0.0592,4.2,0.03,130668.0,175.100,226791.0,46519.0,264.6000,110.660004,0.043339,0.046549,0.068873,0.054401,0.046476,0.026323,0.014632,0.014644,0.017016,0.028707,0.028694,12861120.0,11243350.0,9524725.00,-30.596683,-0.000755,0.055884,0.006549,0.037322,0.011656
252,2001-01-22,19.060,19.62,18.440,19.25,0.292983,9273700.0,0.0,1.0,-0.0097,0.0,0.0525,0.0475,0.0050,32.21,167108.0,0.0600,4.2,0.03,130668.0,175.100,226791.0,46519.0,266.7000,110.239998,-0.012821,0.063991,0.061658,0.054469,0.046250,0.017599,0.014751,0.017688,-0.030420,-0.027571,-0.030509,12764440.0,11309900.0,9884240.00,-30.204418,0.007937,0.002802,-0.003795,0.037322,0.011656
253,2001-01-23,19.310,20.94,19.060,20.50,0.312008,15705900.0,0.0,1.0,-0.0097,0.0,0.0530,0.0482,0.0048,31.66,167108.0,0.0597,4.2,0.03,130668.0,175.100,226791.0,46519.0,266.3000,110.239998,0.064935,0.098636,0.054787,0.052003,0.046681,0.024035,0.017513,0.015690,0.040900,0.047422,0.049245,13107120.0,11500920.0,9843165.00,-32.165744,-0.001500,-0.017075,0.000000,0.037322,0.011656
254,2001-01-24,20.620,20.69,19.560,20.50,0.312008,12805200.0,0.0,1.0,-0.0097,0.0,0.0533,0.0481,0.0052,31.46,167108.0,0.0605,4.2,0.03,130668.0,175.100,226791.0,46519.0,264.7000,111.680000,0.000000,0.057771,0.054628,0.054026,0.047021,0.037837,0.022846,0.020050,-0.037837,-0.022846,-0.020050,15155020.0,12404190.0,10300920.00,-32.165744,-0.006008,-0.006317,0.013062,0.037322,0.011656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5681,2022-11-16,149.130,149.87,147.290,148.79,148.790000,64218266.0,0.0,1.0,1.2900,0.0,0.0367,0.0435,-0.0068,85.62,273451.0,0.0383,3.7,0.05,154369.0,298.012,597492.0,60082.0,1773.7700,106.279999,-0.008331,0.017516,0.044560,0.037662,0.033453,0.016286,-0.001387,0.003215,-0.024617,-0.006945,-0.011546,89384549.0,94301465.3,93300813.65,115.341085,-0.002519,-0.014389,-0.001128,0.072186,-0.001197
5682,2022-11-17,146.430,151.48,146.150,150.72,150.720000,80389400.0,0.0,1.0,1.2900,0.0,0.0377,0.0443,-0.0066,81.69,273451.0,0.0383,3.7,0.05,154369.0,298.012,597492.0,60082.0,1760.8700,106.690002,0.012971,0.036469,0.045745,0.037334,0.033515,0.013785,-0.000465,0.002328,-0.000813,0.013437,0.010644,84246506.8,92685357.4,91554896.45,116.837209,-0.007273,-0.045900,0.003858,0.072186,-0.001197
5683,2022-11-18,152.305,152.70,149.970,151.29,151.290000,74829573.0,0.0,1.0,1.2900,0.0,0.0382,0.0451,-0.0069,80.07,273451.0,0.0383,3.7,0.05,154369.0,298.012,597492.0,60082.0,1749.7400,106.930000,0.003782,0.018204,0.037876,0.035143,0.033596,0.023017,0.004562,0.002938,-0.019235,-0.000780,0.000844,85340828.0,91363835.1,92486449.45,117.279070,-0.006321,-0.019831,0.002249,0.072186,-0.001197
5684,2022-11-21,150.160,150.37,147.715,148.01,148.010000,58724070.0,0.0,1.0,1.2900,0.0,0.0383,0.0448,-0.0065,79.74,273451.0,0.0383,3.7,0.05,154369.0,298.012,597492.0,60082.0,1737.8081,107.839996,-0.021680,0.017974,0.00